# Beta-NMF for GPGPU
In this example the Beta-NMF is used to train dictionnary on a dataset X and project some test data Y on the dictionnary trained on X.

### Links
Source code is available at https://github.com/rserizel/beta_nmf

Dcoumentation is available at http://rserizel.github.io/beta_nmf/

## Import packages
If you plan on using GPGPU make sure you set the Theano flags correctly (see [Theano documentation]).

[Theano documentation]: http://deeplearning.net/software/theano/ 

In [ ]:
from beta_nmf import BetaNMF
from base import nnrandn
import numpy as np

## Training data
Generate random nonnegative data to decompose. At this stage you can decide to load your own data.

Note that in order to notice the impact of using a GPGPU you should use much larger matrices. For a more extensive study of the advantage of using GPGPU for NMF please refer to:

> R. Serizel, S. Essid, and G. Richard. “Mini-batch stochastic approaches for accelerated multiplicative updates in nonnegative matrix factorisation with beta-divergence”. Accepted for publication In *Proc. of MLSP*, p. 5, 2016.

In [ ]:
X = nnrandn((5000, 200))

## Create the BetaNMF
Create a BetaNMF object. Theano variable are initialised based on the shape of the data X.

In [ ]:
nmf = BetaNMF(X.shape, n_iter=10, verbose=1)

## Train the model
Decompose the data X into two nonnegative matrices W and H. Perform 10 iterations and compute the score for each iteration.

In [ ]:
nmf.fit(X)

## Use the model
Generate test data Y and project Y on the dictionnary W learnt on X.

In [ ]:
Y = nnrandn((500, 200))
H = nmf.transform(Y)

## Use custom init values
You can set W and H to custom values with Theano function `set_value()`. Then you should you warm start mode to avoid aoverwritting these values().

For example, below we set H to a custom value and start projection on W with this value as intial factor H.

In [ ]:
nmf.h.set_value(np.ones(H.shape))
H = nmf.transform(Y, warm_start=True)

## Note on the factors shape
In cold start mode the factors are initialised randomly to the correct shape. When using warm start, you should make sure that the shape of the factors is consistent with the data shape. See examples below.
### This will work
Resume projection of Y at iteration 10: data shapes are consistent.

Note also that the initial cost is equal to the final cost in the previous cell which is the expected behaviour here.

In [ ]:
H = nmf.transform(Y, warm_start=True)

### This will not work
Project X starting from a projection of Y: data shapes are not consistent X.shape = (5000,200) and H.shape = (500, 50)

In [ ]:
H = nmf.transform(X, warm_start=True)